# Welcome to RAD-MMM inference tutorial

Before we begin, please download the following dependencies:

1. Language dictionaries from [here](https://drive.google.com/drive/folders/1woNCODwXh9aHu7Fd6b4Jo42aL7f5RFZg) and place them in `RAD-MMM/assets` folder.
2. Download RAD-MMM checkpoint(s) and its config - [decoder.ckpt](https://drive.google.com/file/d/1ZLFHY5iSMdK852UwF1RqFr7cY2ejzeOw/view), [attribute_model.ckpt](https://drive.google.com/file/d/1EduYNwgtRlezJt6RiXMLBBSSOpIbp2CT/view?usp=sharing) and [config.yaml](https://drive.google.com/file/d/1c_dGA82k2Ow65P0vXwYwRTEipNdzTsTa/view?usp=sharing).
3. Download HiFi-GAN vocoder checkpoint and its config - [g_00072000](https://drive.google.com/file/d/1VaH5_MhAjAjHlihi2k-lcOOoy4NqtRV4/view) and [config_16khz.json](https://drive.google.com/file/d/1-eBTNfIh-LSstNirQawHW4jsI-t01jTU/view?usp=sharing).

In [1]:
# imports
import pytorch_lightning as pl
import sys
import yaml
sys.path.append('/akshit/scratch/RAD-MMM/vocoders')
sys.path.append('/akshit/scratch/RAD-MMM')
from pytorch_lightning.cli import LightningCLI
from tts_lightning_modules import TTSModel
from data_modules import BaseAudioDataModule
from jsonargparse import lazy_instance
from decoders import RADMMMFlow
from loss import RADTTSLoss
import inspect
from pytorch_lightning.callbacks import ModelCheckpoint
from training_callbacks import LogDecoderSamplesCallback, \
    LogAttributeSamplesCallback
from utils import get_class_args
from tts_text_processing.text_processing import TextProcessing
from common import Encoder
import torch
import IPython.display as ipd

In [2]:
# Set paths for downloaded files
attribute_model_path = "../generator_ckpt/radmmm_public/attribute_model.ckpt"
gen_config_path = "../generator_ckpt/radmmm_public/config.yaml"
decoder_model_path = "../generator_ckpt/radmmm_public/decoder.ckpt"
voc_model_path = "../generator_ckpt/hfg_public/g_00072000"
voc_config_path = "../generator_ckpt/hfg_public/config_16khz.json"
phonemizer_cfg='{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}'

## Load the model

In [3]:
# load the config

with open(gen_config_path, "r") as f:
    gen_config = yaml.safe_load(f)

In [4]:
def instantiate_class(init):
    """Instantiates a class with the given args and init.

    Args:
        args: Positional arguments required for instantiation.
        init: Dict of the form {"class_path":...,"init_args":...}.

    Returns:
        The instantiated class object.
    """
    kwargs = init.get("init_args", {})
    class_module, class_name = init["class_path"].rsplit(".", 1)
    module = __import__(class_module, fromlist=[class_name])
    args_class = getattr(module, class_name)
    return args_class(**kwargs)

In [5]:
# instantiate submodules

gen_config["model"]["add_bos_eos_to_text"] = False
gen_config["model"]["append_space_to_text"] = True
gen_config["model"]["decoder_path"] = decoder_model_path
gen_config["model"]["encoders_path"] = decoder_model_path
gen_config["model"]["handle_phoneme"] = "word"
gen_config["model"]["handle_phoneme_ambiguous"] = "ignore"
gen_config["model"]["heteronyms_path"] = "tts_text_processing/heteronyms"
gen_config["model"]["output_directory"] = "tutorials/run1"
gen_config["model"]["p_phoneme"] = 1
gen_config["model"]["phoneme_dict_path"] = "tts_text_processing/cmudict-0.7b"
gen_config["model"]["phonemizer_cfg"] = phonemizer_cfg
gen_config["model"]["prediction_output_dir"] = "tutorials/out1"
gen_config["model"]["prepend_space_to_text"] = True
gen_config["model"]["sampling_rate"] = 16000
gen_config["model"]["symbol_set"] = "radmmm_phonemizer_marker_segregated"
gen_config["model"]["vocoder_checkpoint_path"] = voc_model_path
gen_config["model"]["vocoder_config_path"] = voc_config_path

hparams = gen_config["model"]
ttsmodel_kwargs={}
for k,v in hparams.items():
    if type(v) == dict and 'class_path' in v:
        print(k)
        ttsmodel_kwargs[k] = instantiate_class(v)
    elif k != "_instantiator":
        ttsmodel_kwargs[k] = v

decoder


/akshit/scratch/RAD-MMM/common.py:557: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2425.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/opt/conda/lib/python3.10/site-packages/torch/functional.py:1728: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_p

decoder_loss
text_encoder
Applying spectral norm to text encoder LSTM
f0_predictor
f0_predictor_loss
energy_predictor
energy_predictor_loss
voiced_predictor
voiced_predictor_loss
duration_predictor
duration_predictor_loss
speaker_embed_regularization_loss
speaker_accent_cross_regularization_loss


In [6]:
# load the model from checkpoint
model2 = TTSModel.load_from_checkpoint(checkpoint_path=attribute_model_path,\
                                      **ttsmodel_kwargs)


loading:  assets/en_US_word_ipa_map.txt
loading:  assets/es_MX_word_ipa_map.txt
loading:  assets/de_DE_word_ipa_map.txt
loading:  assets/en_UK_word_ipa_map.txt
loading:  assets/es_CO_word_ipa_map.txt
loading:  assets/es_ES_word_ipa_map.txt
loading:  assets/fr_FR_word_ipa_map.txt
loading:  assets/hi_HI_word_ipa_map.txt
loading:  assets/pt_BR_word_ipa_map.txt
loading:  assets/te_TE_word_ipa_map.txt
Number of symbols: 439
updating the speakers set: 7
Initializing f0 predictor
ConvLSTMLinearDAP(
  (bottleneck_layer): BottleneckLayer(
    (projection_fn): ConvNorm(
      (conv): Conv1d(520, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (non_linearity): LeakyReLU(negative_slope=0.01)
  )
  (feat_pred_fn): ConvLSTMLinear(
    (dropout): Dropout(p=0.5, inplace=False)
    (convolutions): ModuleList(
      (0): ConvNorm(
        (conv): Conv1d(48, 256, kernel_size=(15,), stride=(1,), padding=(7,))
      )
      (1-2): 2 x ConvNorm(
        (conv): Conv1d(256, 256, kernel_size=(15,),

## Initialize the datamodule

In [7]:
# initialize the datamodule

gen_config["data"]["inference_transcript"]="model_inputs/resynthesis_prompts.json" #ToDo
gen_config["data"]["batch_size"]=1
gen_config["data"]["phonemizer_cfg"]=phonemizer_cfg
data_module = BaseAudioDataModule(**gen_config['data'])
data_module.setup(stage = "predict")

loading:  assets/en_US_word_ipa_map.txt
loading:  assets/es_MX_word_ipa_map.txt
loading:  assets/de_DE_word_ipa_map.txt
loading:  assets/en_UK_word_ipa_map.txt
loading:  assets/es_CO_word_ipa_map.txt
loading:  assets/es_ES_word_ipa_map.txt
loading:  assets/fr_FR_word_ipa_map.txt
loading:  assets/hi_HI_word_ipa_map.txt
loading:  assets/pt_BR_word_ipa_map.txt
loading:  assets/te_TE_word_ipa_map.txt
Number of symbols: 439
{'basedir': '/home/dcg-adlr-rbadlani-data/multilingual-dataset/opensource/LJSpeech-1.0', 'sampling_rate': '16khz', 'filelist_basedir': 'datasets/opensource/', 'filelist': 'LJSpeech/ljs_audiopath_text_sid_emotion_duration_train_filelist_phonemized.txt', 'language': 'en_US', 'phonemized': True}
processing file: datasets/opensource/LJSpeech/ljs_audiopath_text_sid_emotion_duration_train_filelist_phonemized.txt
{'basedir': '/home/dcg-adlr-rbadlani-data/multilingual-dataset/opensource/HUI-Audio-Corpus-German/Bernd_Ungerer', 'sampling_rate': '16khz', 'filelist_basedir': 'datase

## Run Inference

In [8]:
# run the input through the model
def run_inference(text, speaker_id, input_language_id, target_accent_id):
    script = data_module.tp.convert_to_phoneme(text=text, phoneme_dict=data_module.tp.phonemizer_backend_dict[input_language_id])
    print("Converted the text to phonemes: ", script)
    inferData = [{
      "script": script,
      "spk_id": speaker_id,
      "decoder_spk_id": speaker_id,
      "duration_spk_id": speaker_id,
      "energy_spk_id": speaker_id,
      "f0_spk_id": speaker_id,
      "language": target_accent_id,
      "emotion": "other"
    }]
    
    ## set predictset
    data_module.predictset.data = inferData
    
    ## initialize and get the dataloader
    dl = data_module.predict_dataloader()
    
    ## get the first input
    inp1 = next(iter(dl))
    
    ## move the input tensors to GPU
    for k in inp1.keys():
        if type(inp1[k]) == torch.Tensor:
            inp1[k] = inp1[k].to(device="cuda")

    return model2.forward(inp1) 

In [37]:
# first example

text = "Hope you are enjoying our session so far!"
speaker_id = "ljs"
input_language_id = "en_US"
target_accent_id = input_language_id

output_file_path = run_inference(text, speaker_id, input_language_id, target_accent_id)


Converted the text to phonemes:  {h   ˈ   o ʊ   p} {j   u   ː} {ɑ   ː   ɹ} {ɛ   n   d ʒ   ˈ   ɔ ɪ   ɪ   ŋ} {ˌ   a ʊ   ɚ} {s   ˈ   ɛ   ʃ   ə   n} {s   ˈ   o ʊ} {f   ˈ   ɑ   ː   ɹ}!
en_US|{h ˈ o ʊ p} {j u ː} {ɑ ː ɹ} {ɛ n d ʒ ˈ ɔ ɪ ɪ ŋ} {ˌ a ʊ ɚ} {s ˈ ɛ ʃ ə n} {s ˈ o ʊ} {f ˈ ɑ ː ɹ}!
en_US|[70, 302, 92, 266, 95, 0, 72, 130, 304, 0, 164, 304, 240, 0, 178, 88, 51, 277, 302, 168, 214, 214, 153, 0, 303, 40, 266, 177, 0, 111, 302, 178, 260, 176, 88, 0, 111, 302, 92, 266, 0, 68, 302, 164, 304, 240, 1]
ljs-other
ljs-other
{'f0_median': 203.92970275878906, 'f0_mean': 211.856201171875, 'f0_std': 51.960838317871094, 'log_f0_median': 5.317775249481201, 'log_f0_mean': 5.328337669372559, 'log_f0_std': 0.231437548995018, 'energy_mean': 0.9916158318519592, 'energy_std': 0.029046861454844475, 'n_files': 100}
{'script': ['{h   ˈ   o ʊ   p} {j   u   ː} {ɑ   ː   ɹ} {ɛ   n   d ʒ   ˈ   ɔ ɪ   ɪ   ŋ} {ˌ   a ʊ   ɚ} {s   ˈ   ɛ   ʃ   ə   n} {s   ˈ   o ʊ} {f   ˈ   ɑ   ː   ɹ}!'], 'spk_id': tensor([3], device='cuda:0'

In [38]:
ipd.Audio(output_file_path)

In [39]:
# changing language for the same speaker

text = "आशा है कि आप अब तक हमारे सत्र का आनंद ले रहे हैं!"
speaker_id = "ljs"
input_language_id = "hi_HI"
target_accent_id = input_language_id

output_file_path = run_inference(text, speaker_id, input_language_id, target_accent_id)


Converted the text to phonemes:  {ˈ   a   ː   ʃ   a   ː} {h   ɛ   ː} {k   ˈ   ɪ} {ˌ   a   ː   p} {ˈ   ʌ   b} {t   ˌ   ə   k} {h   ə   m   ˌ   a   ː   ɾ   e   ː} {s   ˈ   ʌ   t   ɾ   ə} {k   a   ː} {a   ː   n   ˈ   ʌ   n   d} {l   ˈ   e   ː} {ɾ   ˌ   ə   h   e   ː} {h   ɛ   ̃}!
hi_HI|{ˈ a ː ʃ a ː} {h ɛ ː} {k ˈ ɪ} {ˌ a ː p} {ˈ ʌ b} {t ˌ ə k} {h ə m ˌ a ː ɾ e ː} {s ˈ ʌ t ɾ ə} {k a ː} {a ː n ˈ ʌ n d} {l ˈ e ː} {ɾ ˌ ə h e ː} {h ɛ ̃}!
hi_HI|[302, 40, 304, 260, 40, 304, 0, 70, 178, 304, 0, 73, 302, 214, 0, 303, 40, 304, 95, 0, 302, 269, 43, 0, 113, 303, 176, 73, 0, 70, 176, 86, 303, 40, 304, 252, 64, 304, 0, 111, 302, 269, 113, 252, 176, 0, 73, 40, 304, 0, 40, 304, 88, 302, 269, 88, 51, 0, 85, 302, 64, 304, 0, 252, 303, 176, 70, 64, 304, 0, 70, 178, 324, 1]
ljs-other
ljs-other
{'f0_median': 203.92970275878906, 'f0_mean': 211.856201171875, 'f0_std': 51.960838317871094, 'log_f0_median': 5.317775249481201, 'log_f0_mean': 5.328337669372559, 'log_f0_std': 0.231437548995018, 'energy_mean': 0.991615

In [40]:
ipd.Audio(output_file_path)

In [41]:
# saying hindi in english accent

text = "आशा है कि आप अब तक हमारे सत्र का आनंद ले रहे हैं!"
speaker_id = "ljs"
input_language_id = "hi_HI"
target_accent_id = "en_US"

output_file_path = run_inference(text, speaker_id, input_language_id, target_accent_id)


Converted the text to phonemes:  {ˈ   a   ː   ʃ   a   ː} {h   ɛ   ː} {k   ˈ   ɪ} {ˌ   a   ː   p} {ˈ   ʌ   b} {t   ˌ   ə   k} {h   ə   m   ˌ   a   ː   ɾ   e   ː} {s   ˈ   ʌ   t   ɾ   ə} {k   a   ː} {a   ː   n   ˈ   ʌ   n   d} {l   ˈ   e   ː} {ɾ   ˌ   ə   h   e   ː} {h   ɛ   ̃}!
en_US|{ˈ a ː ʃ a ː} {h ɛ ː} {k ˈ ɪ} {ˌ a ː p} {ˈ ʌ b} {t ˌ ə k} {h ə m ˌ a ː ɾ e ː} {s ˈ ʌ t ɾ ə} {k a ː} {a ː n ˈ ʌ n d} {l ˈ e ː} {ɾ ˌ ə h e ː} {h ɛ ̃}!
en_US|[302, 40, 304, 260, 40, 304, 0, 70, 178, 304, 0, 73, 302, 214, 0, 303, 40, 304, 95, 0, 302, 269, 43, 0, 113, 303, 176, 73, 0, 70, 176, 86, 303, 40, 304, 252, 64, 304, 0, 111, 302, 269, 113, 252, 176, 0, 73, 40, 304, 0, 40, 304, 88, 302, 269, 88, 51, 0, 85, 302, 64, 304, 0, 252, 303, 176, 70, 64, 304, 0, 70, 178, 324, 1]
ljs-other
ljs-other
{'f0_median': 203.92970275878906, 'f0_mean': 211.856201171875, 'f0_std': 51.960838317871094, 'log_f0_median': 5.317775249481201, 'log_f0_mean': 5.328337669372559, 'log_f0_std': 0.231437548995018, 'energy_mean': 0.991615

In [42]:
ipd.Audio(output_file_path)

In [ ]:
# visualize the output

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
x, sr = librosa.load(output_file_path1)
plt.figure(figsize=(14, 5))
librosa.display.waveshow(x, sr=sr)

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

## Cleanup

In [ ]:
# teardown datamodule
data_module.teardown(stage="predict")

In [ ]:
# free up GPU memory
del model2